In [1]:
import cv2
import numpy as np

DIR_VIDEO_EJEMPLO = './mmnist-medium/batch_0_video_0.mp4'

# Páginas y papers utilizados

### Dataset
* Dataset a utilizar: https://www.kaggle.com/datasets/yichengs/captioned-moving-mnist-dataset-medium-version

### Arquitectura
* Unet 2D: https://lmb.informatik.uni-freiburg.de/people/ronneber/u-net/
* Video explicativo: https://www.youtube.com/watch?v=waIPUsecaaQ

* Temporal smoothing

Cuando tu UNet procesa cada frame por separado, pasa lo siguiente:
El frame 0 → la red lo corrige de UNA forma
El frame 1 → lo corrige levemente distinto
El frame 2 → también distinto

Aunque todos los frames sean parecidos, la red nunca da EXACTAMENTE la misma salida.
Eso genera un efecto feo en video llamado Flickering (parpadeo visual)
El video reconstruido “tiembla”, los colores cambian un poco, las texturas vibran frame a frame.
No es un error del modelo, es una consecuencia de que no ve el tiempo.
El temporal smoothing es una técnica muy simple que suaviza la transición entre frames reconstruidos.

# Visualización del video (sin mancha)

In [2]:
video = cv2.VideoCapture(DIR_VIDEO_EJEMPLO)

if not video.isOpened():
    print("No se pudo abrir el video")
    exit()

while True:
    ret, frame = video.read()
    if not ret:
        break

    # Opcional: ventana completa
    cv2.namedWindow("Video", cv2.WINDOW_NORMAL)
    cv2.setWindowProperty("Video", cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)

    cv2.imshow("Video", frame)

    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

video.release()
cv2.destroyAllWindows()

# Generación de mancha en el vídeo

## Obtener los frames del vídeo
* T → Tiempo (cantidad de frames)
* H → Alto del frame (Height)
* W → Ancho del frame (Width)
* C → Canales de color (Channels)
    * 0 → Red
    * 1 → Green
    * 2 → Blue

In [3]:
cap = cv2.VideoCapture(DIR_VIDEO_EJEMPLO)

frames = []

while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frames.append(frame)

cap.release()

frames = np.array(frames)  # shape: (T, H, W, 3)
print(frames.shape)

(1200, 72, 128, 3)


## Generación de la mancha

In [4]:
def generate_circular_stain(h, w, radius=25, opacity=1, hardness=0.8):
    # Centro aleatorio
    cx = np.random.randint(radius, w - radius)
    cy = np.random.randint(radius, h - radius)

    y, x = np.ogrid[:h, :w]
    dist = np.sqrt((x - cx)**2 + (y - cy)**2)

    # Distancia normalizada 0..1
    norm = dist / radius

    # "hardness" controla qué tan abrupto es el borde
    mask = np.clip(1 - norm, 0, 1)**hardness

    # opacidad de la mancha
    mask = mask * opacity

    return mask[..., None]

def apply_stain_to_video(frames, stain_mask):
    stained = []
    for frame in frames:
        frame_f = frame / 255.0
        stain = np.ones_like(frame_f) * stain_mask

        # Combinar (mancha oscurece + teñido)
        corrupted = frame_f * (1 - stain_mask) + (0.3 * stain)
        corrupted = np.clip(corrupted, 0, 1)
        
        stained.append((corrupted * 255).astype(np.uint8))

    return np.array(stained)


def save_video(frames, filename="video_con_manchas.mp4", fps=24):
    h, w = frames[0].shape[:2]
    writer = cv2.VideoWriter(
        filename,
        cv2.VideoWriter_fourcc(*"mp4v"),
        fps,
        (w, h)
    )
    for f in frames:
        writer.write(cv2.cvtColor(f, cv2.COLOR_RGB2BGR))
    writer.release()


In [5]:
frames

# Genero una mancha circular
mask = generate_circular_stain(frames.shape[1], frames.shape[2])

# Aplico la mancha al video
frames_con_manchas = apply_stain_to_video(frames, mask)

# Guardo el video con manchas
save_video(frames_con_manchas, filename="video_con_manchas_circulares_ejemplo.mp4", fps=24)

# Visualización del video con la mancha

In [6]:
video_mancha_circular = cv2.VideoCapture("./video_con_manchas_circulares_ejemplo.mp4")

if not video_mancha_circular.isOpened():
    print("No se pudo abrir el video")
    exit()

while True:
    ret, frame = video_mancha_circular.read()
    if not ret:
        break

    # Opcional: ventana completa
    cv2.namedWindow("Video", cv2.WINDOW_NORMAL)
    cv2.setWindowProperty("Video", cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)

    cv2.imshow("Video", frame)

    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

video_mancha_circular.release()
cv2.destroyAllWindows()

# Conclusión
Se le generó al video original una mancha y por el alcance de este proyecto y el tipo de video que queremos procesar conviene acortar el video a un clip que sea de entre 5 y 10 segundos ya que la mancha no se va a mover. Esto evitaria ocupar menos espacio para el almacenamiento del dataset y también la velocidad de entrenamiento del modelo, ya que cada video tendría menos frames para analizar que los originales.